## Predicting Final Round Team Compositions in Rainbow 6: Siege

#### Description of the dataset
The data used is an official datadump from Ubisoft, the developers of the game. This data was released after the fifth competitive season of gameplay (In keeping with the games' story as a counter-terrorism first-person shooter, the seasons have operation names. This particular one was named "Operation Velvet Shell"). For the project, the full datadump, featuring a detailed round-by-round breakdown of matches, was used. Each round is detailed from the perspective of each involved player, and includes details on the operator used, the exact loadout of the operator in that round, general measures of the player's skill, and specific match performance statistics, such as the number of kills, and whether they were killed in the round.

The data will be aggregated and reshaped in order to combine all per-player entries into a single entry per round, detailing the choices and statistics of all involved players. These per-round entries will be further aggregated into per game entries, detailing the choices and statistics of all involved players, in all rounds of that game. This aggregated form will the final data used for training and predictions.

#### Description of the project
Using the aggregated data described above, a Generative Adversarial Network (GAN) will be trained to predict the team compositions of the Blue and Orange teams in the final round of a match. Since there are a large number of factors affecting a player's choice of operator in the game, such as personal preference, player skill with specific ops, the choices of other team members, and counterpicks to opponent's favourites, GANs are perfect for this role.

#### Description of GANs
TODO, also cite one of the neat articles on the subject to impress Dr. Miller more when the network inevitably fails to work properly.

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

import matplotlib.pyplot as plt